In [1]:
from pandas import DataFrame
from random import randint, choices, sample, shuffle
from functools import reduce
from operator import add
from math import ceil
import unittest

In [2]:
class Parameters:
    """
        Specify the customizable parameters for the program.
        
        Args: optional
            initial_population : The size of the initial population to begin with. Should be a positive
                                 integer greater than zero.
            mutation_rate: The rate of mutation specified during initialization. Allowed range 1 - 100.

        Return:
            None
            
    """
    def __init__(self, initial_population=100, mutation_rate=1):
        
        self._size = 8
        if initial_population > 0:
            self._initial_population = int(initial_population)
        else:
            raise AttributeError("Initial population should be an integer greater than 0")
        if -1 < mutation_rate < 101:
            self._mutation_rate = int(mutation_rate)
        else:
            raise AttributeError("Mutation rate should be an integer between 1 and 100")
        self._perfect_fitness_score = (4 * self.size) + self.size + 1
        
    @property
    def size(self):
        return self._size
    
    @property
    def initial_population(self):
        return self._initial_population
    
    @property
    def mutation_rate(self):
        return self._mutation_rate
    
    @property
    def perfect_fitness_score(self):
        return self._perfect_fitness_score
     
    def __str__(self):
        return f'Board Size : {self.size} \
                 Initial Population : {self.initial_population} \
                 Mutation Rate : {self.mutation_rate}'
   
    def __repr__(self):
        return f'Parameters Class Instance, \
                 Board Size : {self.size} \
                 Initial Population : {self.initial_population} \
                 Mutation Rate : {self.mutation_rate}'
    
    

In [3]:
class DNA:
    """
        This class represent the DNA of each individual in the population and 
        associated operations that can be performed on that individual
    """
    
    def __init__(self):
        """gene is a sequence of numbers representing the individual in the solution space
            in the range 0-7.
        """
        self._gene = ''.join(map(str, choices(list(range(c.size)), k=c.size)))

        
    @property
    def gene(self):
        return self._gene

    
    @gene.setter
    def gene(self, value):
        if len(value) > c.size:
            raise AttributeError(f"Lenght of the gene cannot be greater than {c.size}")
        else:
            self._gene = value

            
    def north_west_collision(self, df, row, col):
        """
            Args: 
                self: implicit DNA object
                df : DataFrame object used to represent the chess board
                row : row index on the chess board where one of the queen is currently placed.
                col : column index on the chess board where one of the queen is currently placed.
            
            Return:
                Returns True if there are no threats/collision encountered False.
                
            Description:
                From the current position on the chess board checks in the North West direction if
                any threats/collisions are encountered.
        """
        
        flag = True
        while(row > -1 or col > -1):
            row = row - 1
            col = col - 1
            if row > -1 and col > -1:
                if df.iloc[row, col] == 'x':
                    flag = False
                    break
            if row == 0 or col == 0:
                break
        return flag

    
    def north_east_collision(self, df, row, col):
        """
            Args: 
                self: implicit DNA object
                df : DataFrame object used to represent the chess board
                row : row index on the chess board where one of the queen is currently placed.
                col : column index on the chess board where one of the queen is currently placed.
            
            Return:
                Returns True if there are no threats/collision encountered False.
                
            Description:
                From the current position on the chess board checks in the North East direction if
                any threats/collisions are encountered.
        """
        
        flag = True
        while(row > -1 or col < c.size):
            row = row - 1
            col = col + 1
            if row > -1 and col < c.size:
                if df.iloc[row, col] == 'x':
                    flag = False
                    break
            if row == 0 or col == (c.size -1):
                break
        return flag

    
    def south_west_collision(self, df, row, col):
        """
            Args: 
                self: implicit DNA object
                df : DataFrame object used to represent the chess board
                row : row index on the chess board where one of the queen is currently placed.
                col : column index on the chess board where one of the queen is currently placed.
            
            Return:
                Returns True if there are no threats/collision encountered False.
                
            Description:
                From the current position on the chess board checks in the South West direction if
                any threats/collisions are encountered.
        """
        
        flag = True
        while(row < c.size or col > -1):
            row = row + 1
            col = col - 1
            if row < c.size and col > -1:
                if df.iloc[row, col] == 'x':
                    flag = False
                    break
            if row == (c.size - 1) or col == 0:
                break
        return flag
    
    
    def south_east_collision(self, df, row, col):
        """
            Args: 
                self: implicit DNA object
                df : DataFrame object used to represent the chess board
                row : row index on the chess board where one of the queen is currently placed.
                col : column index on the chess board where one of the queen is currently placed.
            
            Return:
                Returns True if there are no threats/collision encountered False.
                
            Description:
                From the current position on the chess board checks in the South East direction if
                any threats/collisions are encountered.
        """
        
        flag = True
        while(row < c.size or col > c.size ):
            row = row + 1
            col = col + 1
            if row < c.size and col < c.size:
                if df.iloc[row, col] == 'x':
                    flag = False
                    break
            if row == (c.size - 1) or col == (c.size - 1):
                break
        return flag
    
    
    def horizontal_collision(self):
        """
            Args: 
                self: implicit DNA object

            Return:
                Returns distinct count of queens in each row.
                
            Description:
                Checks if each row has only one queen placed, add 1 to the score if no 
                collisions/threats are detected in the same row.
        """
        return str(len(set(self.gene)))
    
        
    def adjacency_check(self):
        """
            Args: 
                No explicit argument, DNA object will be the implicit argument
            
            Return:
                Returns '1' if the condition is satisfied else '0'.
                
            Description:
                8 Queens problem solution has a unique character where the queens 
                placed in adjacent columns will never be placed in adjacent rows.
                This function checks for this character in the solution candidate
                and augment the score if it satisfies this character.
        """
        score = 0
        for i in range(c.size):
            if i == 0:
                if (int(self.gene[i]) + 1 == int(self.gene[i+1]) or int(self.gene[i]) - 1 == int(self.gene[i+1])):
                    score += 0
                else:
                    score += 1
    
            if i > 0 and i < c.size - 1:
                if (int(self.gene[i]) + 1 in (int(self.gene[i+1]), int(self.gene[i-1]))) or \
                   (int(self.gene[i]) - 1 in (int(self.gene[i+1]), int(self.gene[i-1]))):
                    score += 0
                else:
                    score += 1
          
            if i == c.size - 1:
                if (int(self.gene[i]) + 1 == int(self.gene[i-1]) or int(self.gene[i]) - 1 == int(self.gene[i-1])):
                    score += 0
                else:
                    score += 1
    
        if score == c.size:
            return str(1)
        else:
            return str(0)
        
        
    def diagonal_collision(self):
        """
            Args:
                self: implicit DNA object.
                row: current row index where one of the queen is placed.
                col: current column index where one of the queen is placed.
            
            Return:
            
            Description:
                This DNA class instance method performs diagonal collision/threat 
                check for a position where the queen is been placed.
            
        
        """
        _score = []
        preliminary_score = []
        df = DataFrame('-' , index=list(range(c.size)), columns=list(range(c.size)))
        mapping = {True:'1', False:'0'}
        
        for i in range(c.size):
            if i < len(self.gene):
                df.iloc[int(self.gene[i]),i] = 'x'
                
        for i in range(len(self.gene)):
            _score = mapping[self.north_west_collision(df, int(self.gene[i]), i)] + \
                     mapping[self.north_east_collision(df, int(self.gene[i]), i)] + \
                     mapping[self.south_west_collision(df, int(self.gene[i]), i)] + \
                     mapping[self.south_east_collision(df, int(self.gene[i]), i)]             
            preliminary_score.append(_score)
            
        return ''.join(preliminary_score)
    
    
    def fitness(self):
        """
            Args:
                self: implicit DNA object.
            
            Return:
                score: Returns the fitness score for an individual candidate.
                
            Description:
                This DNA class instance method runs diagonal collision check,
                horizontal collision check and adjacency check on the DNA 
                object and returns the final score.
        """
        
        score = 0
        temp_score = []
        
        try:
            temp_score.append(self.diagonal_collision())
            temp_score.append(self.horizontal_collision())
            temp_score.append(self.adjacency_check())
        except IndexError:
            print(f'Index Error encountered while parsing {self.gene}')
            
        score = reduce(add,map(int,list(''.join(temp_score))))
        return score  
    
    
    def mutate(self):
        """
            Args:
                self: implicit DNA object.
           
            Return: No explicit return
                
            Description:
            
        
        """
        
        mutation_rate = c.mutation_rate
        n = ceil((mutation_rate * c.size)/100)
        result_set = set(map(str,list(range(c.size))))
        for _ in range(n):
            index = randint(0,c.size -1)
            if len(set(self.gene)) == c.size:
                gene_mutation = list(self.gene)
                e = gene_mutation.pop(index)
                gene_mutation.append(e)
                self.gene = ''.join(gene_mutation)
            else:
                while len(self.gene) < c.size:
                    gene_set = set(self.gene)
                    temp = result_set - gene_set
                    gene_set.add(temp.pop()) #raises KeyError if temp set is empty
                    self.gene = gene_set
    
    
    def crossover(self, other):
        """
            Args:
                self: implicit DNA object.
                other: explicit DNA object.
    
            Return:
                child: modified DNA object
                
            Description:
                This DNA class instance method performs crossover between two DNA objects.
                It creates a collection set of unique DNA genes from the two parents. If the
                If the length of unique DNA genes from both parents fall short of the required
                gene sequence, then one of the parents genes is picked randomly and added to 
                child gene.
        
        """
        new_population_gene_set = set()
        family = (self.gene, other.gene)
        collection = set(self.gene).union(set(other.gene))
        child = DNA()
        child.gene = list(collection)
        for _ in range(c.size - len(child.gene)):
            child.gene.append(family[randint(0, 1)][randint(0,c.size -1)]) # KeyError when set is empty need test case 
        shuffle(child.gene)
        child.gene = ''.join(child.gene)
        child.mutate()
        return child

In [4]:
class Population:
    
    def __init__(self):
        self.population = []
        self.scored_population = []

        
    def generate_initial_population(self):
        """
            Args:
                self: Implicit Population object
            Returns :
                None
            Description: Creates individual DNA objects and add to the population.
        """
        
        while len(self.population) < c.initial_population:
            individual = DNA()
            self.population.append(individual)
            
            
    def score_population(self): 
        """
            Args:
                self: Implicit Population object
            Returns :
                None
            Description: Scores individual DNA objects and add to the scored population list.
        """
                
        self.scored_population = []
        for individual in self.population:
            self.scored_population.append((individual, individual.fitness()))

                
    def generate_mating_pool(self):
        """
            Args:
                self: Implicit Population object
            Returns :
                mating_pool: list object containing individuals from the population. 
            Description: Higher is the fitness greater is the frequency of the individual in the
                            mating pool.
        """
                       
        mating_pool = []
        for individual in self.population:
            n = int((individual.fitness() /c.perfect_fitness_score) * 100)
            for _ in range(n):
                mating_pool.append(individual)
        return mating_pool

In [5]:
class EightQueensProblem:
    
    def __init__(self):
        self.generation = 1
        self.required_fitness = 0
    
    def main(self):
        """
            Args:
                self: implicit EightQueensProblem object.
                
            Return: No explicit return, prints the final solution
                    
            Description: Creates a population instance, generates initial sample
                         population, iterate using a while loop until an individual
                         with perfect fitness score is found.=
        """
        
        p = Population()
        p.generate_initial_population()
        while self.required_fitness != c.perfect_fitness_score:
            max_fitness = 0
            p.score_population()
            fitness_check = p.scored_population
            for i in range(len(fitness_check)):
                if fitness_check[i][1] > max_fitness:
                    max_fitness = fitness_check[i][1]
                    element = fitness_check[i][0]
                if max_fitness == c.perfect_fitness_score:
                    self.required_fitness = c.perfect_fitness_score
                    break
            
            print(f'Generation : {self.generation}, Required Score : {c.perfect_fitness_score}, Generation Best Fitness Score: {max_fitness}, Individual: {element.gene}')
            if max_fitness != c.perfect_fitness_score:
                mating_pool = p.generate_mating_pool()
                new_population = []
                for _ in range(len(p.population)):
                    parent1 = mating_pool[randint(0,len(mating_pool) - 1)]
                    parent2 = mating_pool[randint(0,len(mating_pool) - 1)]
                    child = parent1.crossover(parent2)
                    new_population.append(child)
                p.population = new_population
                self.generation += 1

##### PROGRAM EXECUTION BEGINS

In [6]:
if __name__ == '__main__':
    global c
    c = Parameters(initial_population=100, mutation_rate=40)
    init = EightQueensProblem()
    init.main()

Generation : 1, Required Score : 41, Generation Best Fitness Score: 37, Individual: 57473064
Generation : 2, Required Score : 41, Generation Best Fitness Score: 39, Individual: 15206316
Generation : 3, Required Score : 41, Generation Best Fitness Score: 38, Individual: 05741326
Generation : 4, Required Score : 41, Generation Best Fitness Score: 41, Individual: 73025164


##### UNIT TESTING 

In [7]:
class Chess8QParameterTest(unittest.TestCase):
    
    def setUp(self):
        self.param = Parameters()
        
    def test_program_defaults(self):
        self.assertEqual(100, self.param.initial_population)
        self.assertEqual(1, self.param.mutation_rate)
        
    def test_negative_initial_population(self):
        with self.assertRaises(AttributeError):
            self.param.initial_population = -100
    
    def test_mutation_range(self):
        with self.assertRaises(AttributeError):
            self.param.mutation_rate = 105
    
    def test_edit_read_only_parameters(self):
        with self.assertRaises(AttributeError):
            self.param.initial_population = 10
            self.param.mutation_rate = 4

            
class Chess8QDNATest(unittest.TestCase):
    
    def setUp(self):
        self.param = Parameters()
        self.dna = DNA()
        
    
    def test_gene_sequence_length(self):
        self.assertEqual(len(self.dna.gene),self.param.size)
        
    
    def test_gene_assignment(self):
        with self.assertRaises(AttributeError):
            self.dna.gene = '0123456789'
        
    def test_diagonal_collision(self):
        self.dna.gene = '16257403'
        self.assertEqual(32, reduce(add, map(int,list(self.dna.diagonal_collision()))))

    def test_horizontal_collision(self):
        self.dna.gene = '16257403'
        self.assertEqual('8',self.dna.horizontal_collision())
        
    def test_adjacency_check(self):
        self.dna.gene = '16257403'
        self.assertEqual('1', self.dna.adjacency_check())
        
class Chess8QPopulationTest(unittest.TestCase):
    
    def setUp(self):
        self.param = Parameters()
        self.p = Population()
        
    def test_generate_initial_population(self):
        self.p.generate_initial_population()
        self.assertEqual(self.param.initial_population, len(self.p.population))
        self.assertIsInstance(self.p.population[0],DNA)
        
    def test_score_initial_population(self):
        self.p.score_population()
        self.p.generate_initial_population()
        self.p.score_population()
        self.assertTrue(len(self.p.population),len(self.p.scored_population))
         
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_adjacency_check (__main__.Chess8QDNATest) ... ok
test_diagonal_collision (__main__.Chess8QDNATest) ... ok
test_gene_assignment (__main__.Chess8QDNATest) ... ok
test_gene_sequence_length (__main__.Chess8QDNATest) ... ok
test_horizontal_collision (__main__.Chess8QDNATest) ... ok
test_edit_read_only_parameters (__main__.Chess8QParameterTest) ... ok
test_mutation_range (__main__.Chess8QParameterTest) ... ok
test_negative_initial_population (__main__.Chess8QParameterTest) ... ok
test_program_defaults (__main__.Chess8QParameterTest) ... ok
test_generate_initial_population (__main__.Chess8QPopulationTest) ... ok
test_score_initial_population (__main__.Chess8QPopulationTest) ... ok

----------------------------------------------------------------------
Ran 11 tests in 0.559s

OK
